## Finetuning DistilBERT using SQuAD 2.0

This notebook contains the following fine-tuning groups:

1) No Fine-tuning, testing against SQuAD with pre-trained DistilBERT  (i.e. Baseline)

2) Fine-tuning using data_2 (Traditional split, 80%/10%/10%)

The starting-point for code in this file was found in the Medium blog post titled, Question Answering with DistilBERT (https://medium.com/@sabrinaherbst/question-answering-with-distilbert-ba3e178fdf3d). Main differences include:

 - The DistilBERT model was pre-trined using SQuAD 2.0, rather than SQuAD 1.0
 - DistilBERT performance on SQuAD test data was captured prior to fine-tuning (i.e. Baseline)
  - Exploring traditional split that included unseen test data  (i.e. not validation data)
 - Addition of adhoc dropout rate setting
 - Additional stats for failed results

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load libraries

from transformers import DistilBertModel, DistilBertForMaskedLM, DistilBertConfig, \
            DistilBertTokenizerFast, AutoTokenizer, BertModel, BertForMaskedLM, BertTokenizerFast, BertConfig
from torch import nn
from pathlib import Path
import torch
import pandas as pd
from typing import Optional
from tqdm.auto import tqdm
from torch.optim import AdamW, RMSprop
import numpy as np
from pathlib import Path

import sys
sys.path.append('/content/drive/MyDrive/distilBERT_SQuAD2_w266Project')
from qa_model import QuestionDistilBERT, SimpleQuestionDistilBERT, ReuseQuestionDistilBERT, Dataset, test_model
from util import eval_test_set, count_parameters, print_test_set_incorrect_predictions, \
            analyze_test_set_statistics, analyze_test_set_performance
from my_distilbert import QADataset

# Load tokenizer


In [ ]:
# Load DistilBERT tokenizer, use uncased (lowercase) vocabulary

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

# Load data_2 (Traditional split)

Note: Dataset loader is the "Variable-Length Trunc" described in the report

In [ ]:
# Get paths for all SQuAD dataset text files in training directory -- data_2, Traditional Split
squad_paths_2 = [str(x) for x in Path('/content/drive/MyDrive/distilBERT_SQuAD2_w266Project/data_2/training_squad').glob('**/*.txt')]

# Create training dataset using only SQuAD data, DataLoader with batch size of 8
dataset_2 = Dataset(squad_paths = [str(x) for x in Path('/content/drive/MyDrive/distilBERT_SQuAD2_w266Project/data_2/training_squad').glob('**/*.txt')],
                  natural_question_paths=None,
                  hotpotqa_paths=None, tokenizer=tokenizer)
loader_2 = torch.utils.data.DataLoader(dataset_2, batch_size=8, shuffle=True)
print(f"Approximate Training Dataset Size: {len(dataset_2)}")

## load the validation dataset -- used to be labeled as "test", test_dataset changed to val_dataset, test_loader changed to val_loader
val_dataset_2 = Dataset(squad_paths = [str(x) for x in Path('/content/drive/MyDrive/distilBERT_SQuAD2_w266Project/data_2/validation_squad').glob('**/*.txt')],
                       natural_question_paths=None,
                       hotpotqa_paths = None, tokenizer=tokenizer)
val_loader_2 = torch.utils.data.DataLoader(val_dataset_2, batch_size=4)
print(f"Approximate Validation Dataset Size: {len(val_dataset_2)}")

## load the test dataset -- test_dataset and test_loader should not be used during training
test_dataset_2 = Dataset(squad_paths = [str(x) for x in Path('/content/drive/MyDrive/distilBERT_SQuAD2_w266Project/data_2/test_squad').glob('**/*.txt')],
                       natural_question_paths=None,
                       hotpotqa_paths = None, tokenizer=tokenizer)
test_loader_2 = torch.utils.data.DataLoader(test_dataset_2, batch_size=4)
print(f"Approximate Test Dataset Size: {len(test_dataset_2)}")

Approximate Training Dataset Size: 113000
Approximate Validation Dataset Size: 14000
Approximate Test Dataset Size: 14000


## Model_0
Baseline -- Evaluate the untrained DistilBERT model on the SQuAD 2.0 dataset

In [ ]:
# Load pre-trained DistilBERT model for masked language modeling
model_0 = DistilBertForMaskedLM.from_pretrained("distilbert-base-uncased")

# Get model configuration (architecture, hyperparameters)
config_0 = DistilBertConfig.from_pretrained("distilbert-base-uncased")

# Extract base DistilBERT model without MLM head
mod_0 = model_0.distilbert

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model_0 = SimpleQuestionDistilBERT(mod_0)
model_0.set_dropout_rate(0.15)
model_0.to(device)

# Verify the dropout rates for each layer
for name, module in model_0.named_modules():
    if isinstance(module, nn.Dropout):
        print(f"{name}: {module.p}")

set_dropout_rate
distilbert.embeddings.dropout: 0.15
distilbert.transformer.layer.0.attention.dropout: 0.15
distilbert.transformer.layer.0.ffn.dropout: 0.15
distilbert.transformer.layer.1.attention.dropout: 0.15
distilbert.transformer.layer.1.ffn.dropout: 0.15
distilbert.transformer.layer.2.attention.dropout: 0.15
distilbert.transformer.layer.2.ffn.dropout: 0.15
distilbert.transformer.layer.3.attention.dropout: 0.15
distilbert.transformer.layer.3.ffn.dropout: 0.15
distilbert.transformer.layer.4.attention.dropout: 0.15
distilbert.transformer.layer.4.ffn.dropout: 0.15
distilbert.transformer.layer.5.attention.dropout: 0.15
distilbert.transformer.layer.5.ffn.dropout: 0.15
dropout: 0.15


In [ ]:
# Set model to device
model_0 = model_0.to(device)

# Evaluate baseline performance on data_2 test set
print("Evaluating baseline DistilBERT performance on data_2 before fine-tuning:")
eval_test_set(model_0, tokenizer, test_loader_2, device)

# Optional: Print model architecture and parameter count
count_parameters(model_0)

Evaluating baseline DistilBERT performance on data_1 before fine-tuning:


100%|██████████| 2750/2750 [01:05<00:00, 41.97it/s]


Mean EM:  0.22136363636363637
Mean F-1:  0.23756070843244545
Evaluating baseline DistilBERT performance on data_2 before fine-tuning:


100%|██████████| 3500/3500 [01:24<00:00, 41.25it/s]

Mean EM:  0.14314285714285716
Mean F-1:  0.16755100848835633
+---------------------------------------------------------+------------+
|                         Modules                         | Parameters |
+---------------------------------------------------------+------------+
|       distilbert.embeddings.word_embeddings.weight      |  23440896  |
|     distilbert.embeddings.position_embeddings.weight    |   393216   |
|          distilbert.embeddings.LayerNorm.weight         |    768     |
|           distilbert.embeddings.LayerNorm.bias          |    768     |
|  distilbert.transformer.layer.0.attention.q_lin.weight  |   589824   |
|   distilbert.transformer.layer.0.attention.q_lin.bias   |    768     |
|  distilbert.transformer.layer.0.attention.k_lin.weight  |   589824   |
|   distilbert.transformer.layer.0.attention.k_lin.bias   |    768     |
|  distilbert.transformer.layer.0.attention.v_lin.weight  |   589824   |
|   distilbert.transformer.layer.0.attention.v_lin.bias   |    

66364418

# Model_2
Train DistilBERT using SQuAD 2.0 --- data_2, Traditional Split

In [ ]:
# Load pre-trained DistilBERT model for masked language modeling
model_2 = DistilBertForMaskedLM.from_pretrained("distilbert-base-uncased")

# Get model configuration (architecture, hyperparameters)
config_2 = DistilBertConfig.from_pretrained("distilbert-base-uncased")

# Extract base DistilBERT model without MLM head
mod_2 = model_2.distilbert

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model_2 = SimpleQuestionDistilBERT(mod_2)
model_2.set_dropout_rate(0.15)
model_2.to(device)

# Verify the dropout rates for each layer
for name, module in model_2.named_modules():
    if isinstance(module, nn.Dropout):
        print(f"{name}: {module.p}")

set_dropout_rate
distilbert.embeddings.dropout: 0.15
distilbert.transformer.layer.0.attention.dropout: 0.15
distilbert.transformer.layer.0.ffn.dropout: 0.15
distilbert.transformer.layer.1.attention.dropout: 0.15
distilbert.transformer.layer.1.ffn.dropout: 0.15
distilbert.transformer.layer.2.attention.dropout: 0.15
distilbert.transformer.layer.2.ffn.dropout: 0.15
distilbert.transformer.layer.3.attention.dropout: 0.15
distilbert.transformer.layer.3.ffn.dropout: 0.15
distilbert.transformer.layer.4.attention.dropout: 0.15
distilbert.transformer.layer.4.ffn.dropout: 0.15
distilbert.transformer.layer.5.attention.dropout: 0.15
distilbert.transformer.layer.5.ffn.dropout: 0.15
dropout: 0.15


In [ ]:
# Set model to training mode (enables dropout, batch norm, etc.)
model_2.train()

# Initialize RMSprop optimizer and learning rate of 4e-5
optim = RMSprop(model_2.parameters(), lr=4e-5)

In [ ]:
# Train the DistilBERT model with data_2 for 5 epochs
# Validation data used to evaluate performance during training

epochs = 5

for epoch in range(epochs):
   # Training loop
   loop = tqdm(loader_2, leave=True)  # Progress bar for training batches
   model_2.train()  # Set model to training mode
   mean_training_loss = []

   for batch in loop:
       # Zero gradients at start of each batch
       optim.zero_grad()

       # Move batch data to GPU/CPU device
       input_ids = batch['input_ids'].to(device)
       attention_mask = batch['attention_mask'].to(device)
       start = batch['start_positions'].to(device)
       end = batch['end_positions'].to(device)

       # Forward pass
       outputs = model_2(input_ids, attention_mask=attention_mask,
                      start_positions=start, end_positions=end)
       loss = outputs['loss']

       # Backward pass and optimization
       loss.backward()
       optim.step()

       # Track and display training progress
       mean_training_loss.append(loss.item())
       loop.set_description(f'Epoch {epoch}')
       loop.set_postfix(loss=loss.item())
   print("Mean Training Loss", np.mean(mean_training_loss))

   # Validation loop
   loop = tqdm(val_loader_2, leave=True)  # Progress bar for validation batches
   model_2.eval()  # Set model to evaluation mode
   mean_val_loss = []

   for batch in loop:
       # Move batch data to GPU/CPU device
       input_ids = batch['input_ids'].to(device)
       attention_mask = batch['attention_mask'].to(device)
       start = batch['start_positions'].to(device)
       end = batch['end_positions'].to(device)

       # Forward pass (no gradients needed for validation)
       outputs = model_2(input_ids, attention_mask=attention_mask,
                      start_positions=start, end_positions=end)
       loss = outputs['loss']

       # Track and display validation progress
       mean_val_loss.append(loss.item())
       loop.set_description(f'Epoch {epoch} Validation set')
       loop.set_postfix(loss=loss.item())
   print("Mean Validation Loss", np.mean(mean_val_loss))

  0%|          | 0/14125 [00:00<?, ?it/s]

Mean Training Loss 1.2780291521401532


  0%|          | 0/3500 [00:00<?, ?it/s]

Mean Validation Loss 1.4126887596430524


  0%|          | 0/14125 [00:00<?, ?it/s]

...

Mean Validation Loss 1.4424953765503097


  0%|          | 0/14125 [00:00<?, ?it/s]

Mean Training Loss 0.5054844508641193


  0%|          | 0/3500 [00:00<?, ?it/s]

Mean Validation Loss 1.459540368686297


In [ ]:
# Save trained model parameters (weights & biases)
torch.save(model_2.state_dict(), "simple_distilbert_qa_data_2.model")

In [ ]:
# Initialize a new instance of our custom QA model
model_2 = SimpleQuestionDistilBERT(mod_2)

# Load previously saved model parameters from disk
model_2.load_state_dict(torch.load("simple_distilbert_qa_data_2.model"))

<ipython-input-11-2d7a63a66431>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_2.load_state_dict(torch.load("simple_distilbert_qa_data_2.model"))


<All keys matched successfully>

In [ ]:
# Evaulate data_2 performance for using test data
eval_test_set(model_2, tokenizer, test_loader_2, device)

100%|██████████| 3500/3500 [01:11<00:00, 49.01it/s]

Mean EM:  0.6170714285714286
Mean F-1:  0.6931920470180789
